In [ ]:
Author = "Dennis C. Norton"
Collaborators = ["Bruno de Almeida",
                 "Anna Harris",
                 "Maggie Lau",
                 "Fan Ye",
                 "Echo Zhang"
                ]

# National Vehicle Collision Data Preparation

---

**Data Source**: Government of Canada National Collision Database

**Data File**: https://open.canada.ca/data/en/dataset/1eb9eba7-71d1-4b30-9fb1-30cbdab7e63a

**Data Dictionary**: https://open.canada.ca/data/en/dataset/1eb9eba7-71d1-4b30-9fb1-30cbdab7e63a/resource/09b74afc-2745-4382-8a02-3e256c4b28fd 

**Data Licence**: https://open.canada.ca/en/open-government-licence-canada

---

Contains information licensed under the Open Government Licence – Canada.  See below for details.

---

The data captures information about the the nature of collisions, vehicles involved, and people involved.  The data will be used in comparison to Toronto collision data.

This module will prepare the data for analysis by saving the data in three categories:

1.   Case information 
     *   When did the collision occur
     *   What were the circumstances when the collision occured?
     *   How severe was the outcome of the collision?
2.   Vehicle information
     *   What type of vehicle?
     *   What is the year of the vehicle?
3.   People information
     *   Who is the person: gender, age?
     *   Where was the person positioned in the vehicle?
     *   What was the severity of their injuries?


In [ ]:
# Initialize the environment

import pandas as pd
import numpy as np

file_path = './Data Files/'
file_in = 'NCDB_1999_to_2017.csv'
case_file = 'NCDB_Cases.csv'
vehicle_file = 'NCDB_Vehicles.csv'
people_file = 'NCDB_People.csv'

The source data is mainly numeric data and does not contain any null values, but it does contain characters to indicate missing or unknown data.  Converter funtions have been created to read the data into a consistent data type within each column.

C_CASE, V_ID, and P_ID fields will be used for index values in the case_file, vehicle_file, and people_file respectively.  To avoid potential issues later, these fields are converted to string values.

C_MNTH and C_WDAY will be kept in string format so that they can be used as dictionary keys to convert them text values for month and day of the week respectively.

In [ ]:
# Variables and functions to clean the input data

days = {'0':'Mon', '1':'Tue', '2':'Wed', '3':'Thu', '4':'Fri', 
        '5':'Sat', '6':'Sun', 'U':'Unknown', 'N':'Unknown'}
months = {'01':'Jan', '02':'Feb', '03':'Mar', '04':'Apr', 
          '05':'May', '06':'Jun', '07':'Jul', '08':'Aug',
          '09':'Sep', '10':'Oct', '11':'Nov', '12':'Dec',
          'UU':'Unknown', 'NN':'Unknown'}
unknowns = ['N', 'NN', 'NNNN', 'Q', 'U', 'X', 
            'QQ', 'UU', 'XX', 'UUUU', 'XXXX']

def int_cleanup(x):
    if x in unknowns:
        return -1
    else:
        return int(x)

def str_cleanup(x):
    if x in unknowns:
        return 'Unknown'
    else:
        return x

def untouched_cleanup(x):
    return str(x)

# Adjusting date values to be inline with pandas datetime convention.

def day_cleanup(x):
    if x.isnumeric():
        return str(int(x) - 1)
    else:
        return x

# Age is converted to age categories to match the Toronto data

def age_cleanup(x):
    if x in unknowns:
        return 'Unknown'
    else:
        x = int(x)
        return str((x // 5) * 5) + ' to ' + str((x // 5) * 5 + 4)

def pisev_cleanup(x):
    if x.isnumeric():
        return int(x)
    else:
        return 0

# Add a character to each of the ID columns to ensure they remain strings.

def clean_caseID(x):
    return 'C' + str(int(x))

def clean_vehlID(x):
    if x.isnumeric():
        x = str(int(x))
    return 'V' + x

def clean_persID(x):
    if x.isnumeric():
        x = str(int(x))
    return 'P' + x

# Identify the types of vehicles in the incident.  
# Categories are based on Toronto data categories.

def who_else(x):
    result = []
    
    if x['PEDESTRIAN']:
        result.append('Pedestrian')
    if x['CYCLIST']:
        result.append('Cyclist')
    if x['AUTOMOBILE']:
        result.append('Automobile')
    if x['MOTORCYCLE']:
        result.append('Motorcycle')
    if x['TRUCK']:
        result.append('Truck')
    if x['TRSN_CITY_VEH']:
        result.append('City Vehicle')
    if x['EMERG_VEH']:
        result.append('Emergency Vehicle')
    if result == []:
        result.append('Not Recorded')   
    
    # Convert the list to a string to avoid issues with the group_by function later.

    result_str = ', '.join(map(str, result))

    return result_str

In [ ]:
# Read all information into a dataframe

file_content = pd.read_csv(file_path + file_in,
                           nrows = None,
                           converters = {'C_CASE': clean_caseID,
                                         'V_ID': clean_vehlID,
                                         'P_ID': clean_persID,
                                         'C_MNTH': untouched_cleanup,
                                         'C_WDAY': day_cleanup,
                                         'C_HOUR': int_cleanup,
                                         'C_SEV': int_cleanup,
                                         'C_VEHS': int_cleanup,
                                         'C_CONF': int_cleanup,
                                         'C_RCFG': int_cleanup,
                                         'C_WTHR': int_cleanup,
                                         'C_RSUR': int_cleanup,
                                         'C_RALN': int_cleanup,
                                         'C_TRAF': int_cleanup,
                                         'V_TYPE': untouched_cleanup, 
                                         'V_YEAR': int_cleanup, 
                                         'P_SEX': str_cleanup,
                                         'P_AGE': age_cleanup,
                                         'P_PSN': int_cleanup,
                                         'P_ISEV': int_cleanup,
                                         'P_SAFE': int_cleanup,
                                         'P_USER': int_cleanup})

The valid values of the source C_WDAY is a value from 1 to 7 to represent Monday through Sunday.  The day_cleanup() function converted the values to a range of 0 to 6 to match pandas datetime convention for weekdays.  This means that the values no longer match the documentation.  To avoid confusion with the data dictionary, the column will be renamed.  In addition, a new column will be created with the name of the day of the week so that it is clear which day of the week is being referred to.

A new column will also be added for month name, although there is no confusion with the month range of 1 to 12.

In [ ]:
file_content.rename({'C_WDAY': 'pd_WDAY'}, axis=1, inplace=True)
file_content['WDAY_NAME'] = file_content['pd_WDAY'].map(lambda x: days[x])
file_content['MNTH_NAME'] = file_content['C_MNTH'].map(lambda x: months[x])

The Toronto data is for the years 2006 to 2020.  The national data is from 1999 to 2017.  For comparison purposes, we will drop years prior to 2006.

The Toronto data is only for collisions where at least one person was seriously injured or fataly injured.  The national data includes incidents with no injuries.  For comparison purposes, collisions with no injuries will be removed from the national data.

Note, the C_SEV field will indicate whether or not a fatality was involved, but P_ISEV is more detailed and indicates either; no injury, injury, or fatality.  We will use P_SEV to remove incidents with no injuries.

In [ ]:
highest_injury = file_content.groupby(['C_CASE']).aggregate({'P_ISEV': 'max'})

file_content = file_content.loc[file_content['C_YEAR'] >= 2006]
file_content = file_content.loc[file_content['C_CASE'].isin(list(highest_injury.loc[highest_injury['P_ISEV'] > 1].index))]

The following shows information about the resulting dataframe thus far.

In [ ]:
print(file_content.shape)
file_content.head(5)

(3925018, 25)


,C_YEAR,C_MNTH,pd_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER,C_CASE,WDAY_NAME,MNTH_NAME
2847537,2006,01,0,18,2,1,2,-1,1,3,...,P1,M,15 to 19,12,2,13,-1,C1063702,Mon,Jan
2847538,2006,01,0,18,2,1,2,-1,1,3,...,P2,M,15 to 19,11,1,-1,-1,C1063702,Mon,Jan
2847539,2006,01,0,19,2,2,22,-1,1,5,...,P1,F,15 to 19,11,2,1,1,C1063774,Mon,Jan
2847540,2006,01,0,19,2,2,22,-1,1,5,...,P1,M,15 to 19,11,1,-1,1,C1063774,Mon,Jan
2847541,2006,01,0,12,2,4,35,2,2,1,...,P1,M,20 to 24,11,1,-1,1,C1063857,Mon,Jan


In [ ]:
file_content.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3925018 entries, 2847537 to 6772562
Data columns (total 25 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   C_YEAR     int64 
 1   C_MNTH     object
 2   pd_WDAY    object
 3   C_HOUR     int64 
 4   C_SEV      int64 
 5   C_VEHS     int64 
 6   C_CONF     int64 
 7   C_RCFG     int64 
 8   C_WTHR     int64 
 9   C_RSUR     int64 
 10  C_RALN     int64 
 11  C_TRAF     int64 
 12  V_ID       object
 13  V_TYPE     object
 14  V_YEAR     int64 
 15  P_ID       object
 16  P_SEX      object
 17  P_AGE      object
 18  P_PSN      int64 
 19  P_ISEV     int64 
 20  P_SAFE     int64 
 21  P_USER     int64 
 22  C_CASE     object
 23  WDAY_NAME  object
 24  MNTH_NAME  object
dtypes: int64(15), object(10)
memory usage: 778.6+ MB


We engineered columns to describe the type of incident, for example; Auto/Pedestrian, Auto/Truck, Auto, ...

The type of vehicle is described in the V_TYPE data.  The engineered data will be summarized up to the case level.

In [ ]:
file_content['PEDESTRIAN'] = False
file_content['CYCLIST'] = False
file_content['AUTOMOBILE'] = False
file_content['MOTORCYCLE'] = False
file_content['TRUCK'] = False
file_content['TRSN_CITY_VEH'] = False
file_content['EMERG_VEH'] = False
file_content['OTHER_VEH'] = False

file_content.loc[file_content['V_TYPE'].isin(['NN']), 'PEDESTRIAN'] = True
file_content.loc[file_content['V_TYPE'].isin(['17']), 'CYCLIST'] = True
file_content.loc[file_content['V_TYPE'].isin(['01']), 'AUTOMOBILE'] = True
file_content.loc[file_content['V_TYPE'].isin(['14', '16']), 'MOTORCYCLE'] = True
file_content.loc[file_content['V_TYPE'].isin(['05', '06', '07', '08']), 'TRUCK'] = True
file_content.loc[file_content['V_TYPE'].isin(['11']), 'TRSN_CITY_VEH'] = True
file_content.loc[file_content['V_TYPE'].isin(['21']), 'EMERG_VEH'] = True
file_content.loc[file_content['V_TYPE'].isin(['09', '10', '18', '19', 
                                              '20', '22', '23', 'QQ']), 'OTHER_VEH'] = True

case_data = file_content.groupby(['C_CASE', 'C_YEAR', 'MNTH_NAME', 'C_MNTH',
                                  'WDAY_NAME', 'pd_WDAY', 'C_HOUR', 'C_SEV', 
                                  'C_VEHS', 'C_CONF', 'C_RCFG', 'C_WTHR',
                                  'C_RSUR', 'C_RALN', 'C_TRAF'
                                 ]).\
                         aggregate({'PEDESTRIAN': 'max',
                                    'CYCLIST': 'max',
                                    'AUTOMOBILE': 'max',
                                    'MOTORCYCLE': 'max',
                                    'TRUCK': 'max',
                                    'TRSN_CITY_VEH': 'max',
                                    'EMERG_VEH': 'max',
                                    'OTHER_VEH': 'max'
                                   })

case_data['INVOLVED'] = case_data.apply(who_else, axis=1)

Show information about the case data.

In [ ]:
print(case_data.shape)
case_data.head(5)

(1506549, 9)


,,,,,,,,,,,,,,,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,OTHER_VEH,INVOLVED
C_CASE,C_YEAR,MNTH_NAME,C_MNTH,WDAY_NAME,pd_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,,,,,,,,,
C1063686,2006,Unknown,UU,Unknown,U,-1,2,-1,-1,1,-1,-1,-1,-1,True,False,True,False,True,False,False,False,"Pedestrian, Automobile, Truck"
C1063687,2006,Jan,01,Wed,2,7,2,2,35,4,2,5,1,15,False,False,True,False,False,False,False,True,Automobile
C1063688,2006,Jul,07,Sat,5,4,2,1,2,2,1,6,1,18,False,False,True,False,False,False,False,False,Automobile
C1063689,2006,Nov,11,Thu,3,23,2,1,2,2,3,2,1,1,True,False,True,False,False,False,False,False,"Pedestrian, Automobile"
C1063690,2006,Oct,10,Wed,2,20,2,1,2,2,1,1,1,18,False,False,True,False,False,False,False,False,Automobile


In [ ]:
case_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1506549 entries, ('C1063686', 2006, 'Unknown', 'UU', 'Unknown', 'U', -1, 2, -1, -1, 1, -1, -1, -1, -1) to ('C2570235', 2017, 'Jul', '07', 'Thu', '3', 21, 2, 2, 1, 1, 2, 1, 1, 18)
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   PEDESTRIAN     1506549 non-null  bool  
 1   CYCLIST        1506549 non-null  bool  
 2   AUTOMOBILE     1506549 non-null  bool  
 3   MOTORCYCLE     1506549 non-null  bool  
 4   TRUCK          1506549 non-null  bool  
 5   TRSN_CITY_VEH  1506549 non-null  bool  
 6   EMERG_VEH      1506549 non-null  bool  
 7   OTHER_VEH      1506549 non-null  bool  
 8   INVOLVED       1506549 non-null  object
dtypes: bool(8), object(1)
memory usage: 60.3+ MB


In [ ]:
# Write case data to a csv file

case_data.to_csv(file_path + case_file, index = True)

In [ ]:
# Remove duplicates and write vehicle data to a csv file

vehicle_data = file_content.drop_duplicates(subset = ['C_CASE', 'V_ID'])
vehicle_data.to_csv(file_path + vehicle_file, index = True,
                columns = ['C_CASE', 'V_ID', 'V_TYPE', 'V_YEAR'])

In [ ]:
# Write people data to a csv file

file_content.to_csv(file_path + people_file, index = True,
                    columns = ['C_CASE', 'V_ID', 'P_ID', 'P_SEX', 'P_AGE',
                               'P_PSN', 'P_ISEV', 'P_SAFE', 'P_USER'
                              ])

# Open Government Licence - Canada

You are encouraged to use the Information that is available under this licence with only a few conditions.

**Using Information under this licence**

Use of any Information indicates your acceptance of the terms below.
The Information Provider grants you a worldwide, royalty-free, perpetual, non-exclusive licence to use the Information, including for commercial purposes, subject to the terms below.
You are free to:
Copy, modify, publish, translate, adapt, distribute or otherwise use the Information in any medium, mode or format for any lawful purpose.
You must, where you do any of the above:
Acknowledge the source of the Information by including any attribution statement specified by the Information Provider(s) and, where possible, provide a link to this licence.
If the Information Provider does not provide a specific attribution statement, or if you are using Information from several information providers and multiple attributions are not practical for your product or application, you must use the following attribution statement:
Contains information licensed under the Open Government Licence – Canada.

The terms of this licence are important, and if you fail to comply with any of them, the rights granted to you under this licence, or any similar licence granted by the Information Provider, will end automatically.

**Exemptions**

This licence does not grant you any right to use:

Personal Information;
third party rights the Information Provider is not authorized to license;
the names, crests, logos, or other official symbols of the Information Provider; and
Information subject to other intellectual property rights, including patents, trade-marks and official marks.
Non-endorsement
This licence does not grant you any right to use the Information in a way that suggests any official status or that the Information Provider endorses you or your use of the Information.

**No Warranty**

The Information is licensed “as is”, and the Information Provider excludes all representations, warranties, obligations, and liabilities, whether express or implied, to the maximum extent permitted by law.

The Information Provider is not liable for any errors or omissions in the Information, and will not under any circumstances be liable for any direct, indirect, special, incidental, consequential, or other loss, injury or damage caused by its use or otherwise arising in connection with this licence or the Information, even if specifically advised of the possibility of such loss, injury or damage.

**Governing Law**

This licence is governed by the laws of the province of Ontario and the applicable laws of Canada.

Legal proceedings related to this licence may only be brought in the courts of Ontario or the Federal Court of Canada.

**Definitions**

In this licence, the terms below have the following meanings:

"Information"
means information resources protected by copyright or other information that is offered for use under the terms of this licence.
"Information Provider"
means Her Majesty the Queen in right of Canada.
“Personal Information”
means “personal information” as defined in section 3 of the Privacy Act, R.S.C. 1985, c. P-21.
"You"
means the natural or legal person, or body of persons corporate or incorporate, acquiring rights under this licence.
Versioning
This is version 2.0 of the Open Government Licence – Canada. The Information Provider may make changes to the terms of this licence from time to time and issue a new version of the licence. Your use of the Information will be governed by the terms of the licence in force as of the date you accessed the information.